##Install libs

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

##Drive for train and val datasets

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Config

In [2]:
test_data_path = "drive/MyDrive/mipd_test.jsonl"
GRAD_ACCUM = 8
MAX_NEW_TOKENS = 256
BATCH_SIZE = 1
load_from_checkpoint = True
checkpoint = 500
max_seq_length = 16384
base_model_dir = "drive/MyDrive/bielik-4.5b-base"
checkpoint_dir = "drive/MyDrive/unsloth_bielik_4_5B_sft/checkpoint-" + str(checkpoint)

##Load model

In [11]:
from unsloth import FastLanguageModel
from google.colab import userdata
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model_dir,
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    use_gradient_checkpointing = "unsloth",
)

==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory drive/MyDrive/bielik-4.5b-base.

###Add LoRA adapters

In [ ]:
if not load_from_checkpoint:
  model = FastLanguageModel.get_peft_model(
      model,
      r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
      target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj",],
      lora_alpha = 16,
      lora_dropout = 0, # Supports any, but = 0 is optimized
      bias = "none",    # Supports any, but = "none" is optimized
      # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
      use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
      random_state = 3407,
      use_rslora = False,  # We support rank stabilized LoRA
      loftq_config = None, # And LoftQ
  )

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2026.1.2 patched 60 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


##Load datasets

In [12]:
from datasets import load_dataset

train_data_path = "drive/MyDrive/mipd_train.jsonl"
val_data_path = "drive/MyDrive/mipd_val.jsonl"

# Load the datasets
datasets = load_dataset("json", data_files={
    "train": train_data_path,
    "validation": val_data_path
}, streaming=True)

print(datasets)

train = datasets["train"]
val = datasets["validation"]

IterableDatasetDict({
    train: IterableDataset({
        features: Unknown,
        num_shards: 1
    })
    validation: IterableDataset({
        features: Unknown,
        num_shards: 1
    })
})


In [13]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # or "qwen-2.5"
)

# 2. Define the Function for Alpaca Format
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []

    # Iterate over the 3 columns simultaneously
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Convert your 3 columns into the standard list of messages
        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": input},
            {"role": "assistant", "content": output}
        ]

        # Apply the template to turn the list into a string
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        texts.append(text)

    return { "text" : texts }

# 3. Apply it
train_dataset = train.map(formatting_prompts_func, batched=True)
val_dataset = val.map(formatting_prompts_func, batched=True)

NameError: name 'tokenizer' is not defined

##Init trainer

In [ ]:
# Run this once to get your total dataset size
with open(train_data_path, "r") as f:
    total_samples = sum(1 for _ in f)

print(f"Total samples: {total_samples}")

steps_in_epoch = total_samples // (BATCH_SIZE * GRAD_ACCUM)

print("Steps in epoch:", steps_in_epoch)

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    args = SFTConfig(
        dataset_text_field = "text",

        per_device_train_batch_size = BATCH_SIZE,
        gradient_accumulation_steps = GRAD_ACCUM, # Use GA to mimic batch size!
        dataloader_num_workers = 0,

        max_steps = steps_in_epoch*2, #2 epochs

        output_dir = "/content/drive/MyDrive/unsloth_bielik_4_5B_sft",
        save_steps = 100,
        save_total_limit = 3,

        warmup_steps = 50,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
4.498 GB of memory reserved.


##Run sft

In [ ]:
trainer_stats = trainer.train(resume_from_checkpoint = checkpoint_dir)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 21,488 | Num Epochs = 9,223,372,036,854,775,807 | Total steps = 2,686
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 99,778,560 of 4,857,038,848 (2.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.889800
2,0.843000
3,0.817700
4,0.873000
5,0.863000
6,0.231400
7,0.112500
8,0.044900
9,0.032800
10,0.136100


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
message = val[0]

messages = [
    {"role" : "system", "content" : message["instruction"]},
    {"role" : "user", "content" : message["input"]}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1000, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

print("--------")
print("Correct label: ", message["output"])